In [81]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import YoutubeLoader
import streamlit as st
import os

In [82]:

OPENAI_API_KEY = st.secrets["OPENAI_API_KEY"]
youtube_url = "https://www.youtube.com/watch?v=P1iCMyuwOfg&ab_channel=ChefReactions"
youtube_url_2 = "https://www.youtube.com/watch?v=Gm2-Kc-A7Kc&t=2s&ab_channel=ColoradoChannel"
openai_embed_model = "text-embedding-ada-002"
openai_model = "gpt-3.5-turbo-16k"
llm = ChatOpenAI(api_key=OPENAI_API_KEY, model=openai_model, temperature=0.1)

In [83]:
def process_video(url):
    loader = YoutubeLoader.from_youtube_url(url)
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model = "text-embedding-ada-002")
    vector_store = FAISS.from_documents(chunks, embedding)
    return vector_store

In [84]:
vector_store = process_video(youtube_url_2)

In [98]:
query1 = 'give me a bulleted list of the talking points'
query2 = 'what is the most important point of the video?'
query3 = 'please list the bills that were voted on. Give the bill number, the bill name, and the main talking points about each'

In [88]:
def process_question(question):
    retriever = vector_store.as_retriever()
    qa = RetrievalQA.from_chain_type(llm = llm, chain_type='stuff', retriever=retriever)
    results = qa.invoke(question)
    return results


In [99]:
response = submit_question(query3)

In [102]:
print(response['result'])

Here is a list of the bills that were voted on:

1. House Bill 1368 - This bill was passed. It concerns requirements for property and casualty insurers offering insurance policies in the state to provide certain documents to their insurance.

2. House Bill 1381 - This bill was passed. It concerns the continuation of the division of financial services in the department of regulatory agencies and implements recommendations from the 2023 Sunset report.

3. House Bill 1438 - This bill was passed. It concerns the implementation of certain affordable prescription drug programs and makes an appropriation.

4. House Bill 1443 - This bill was passed. It concerns fees to which public trustees are entitled for performing services conferred upon them by statute.

5. Senate Bill 13 - This bill was passed. It concerns the compensation of attorneys working in the office of a district attorney.

6. Senate Bill 55 - This bill was passed. It concerns supporting the behavioral health care of individuals 